In [1]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11,8)})
sns.set(font_scale=1.2)
import numpy as np
from utils import fix_wrong_record_id, fill_id, format_datetime_columns, delete_redundant_columns, mark_inattentive_participants
from merge_datasets import split_to_intersection_and_union, fix_wrong_groups, resolve_contradiction
from columns_index_for_merging import columns_dict, merging_functions_by_columns_set, keep_both_values, weird_text_columns

In [2]:
df_2022 = pd.read_csv("../original_data/ImmiRiskIPT2022_DATA_2022-12-12_1719.csv", na_values= ' ')
df_2021 = pd.read_csv(r"../original_data/Schneider Depression Clinic Database.csv", na_values= ' ')

C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (44,51,53,99,325,366,439,453,459,464,466,501,507,512,514,521,530,532,569,571,580,599,766,771,777,781,786,788,793,795,804,806,826,843,845,854,873,883,918,941,979,991,1002,1040,1045,1047,1049,1051,1053,1054,1080,1096,1119,1122,1126,1127,1131,1132,1133,1134,1136,1137,1138,1141,1142,1144,1161,1183,1186,1237,1239,1259,1261,1262,1263,1269,1270,1271,1272,1384,1387,1389,1405,1408,1411,1415,1432,1434,1435,1436,1442,1443,1444,1445,1499,1568,1569,1572,1573) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
[i for i in df_2021.columns if 'trqsfmaris' in i]

['trqsfmarisclin_timestamp',
 'trqsfmarisclin_complete',
 'trqsfmaris_stu_timestamp',
 'trqsfmaris_stu_complete']

# the preparing datasets

In [3]:
df_2021_1 = fix_wrong_record_id(df_2021, year=2021)
df_2022_1 = fix_wrong_record_id(df_2022, year=2022)

##### fill id function - only for 2022

In [4]:
df_2022_2 = fill_id(df_2022_1)

In [5]:
df_2021_3 = format_datetime_columns(df_2021_1)
df_2022_3 = format_datetime_columns(df_2022_2)

In [6]:
df_2021_4 = mark_inattentive_participants(df_2021_3, year=2021)
df_2022_4 = mark_inattentive_participants(df_2022_3, year=2022)

In [7]:
df_2021_5, df_2022_5 = delete_redundant_columns(df_2021 = df_2021_4, df_2022 = df_2022_4)

# merging the datasets

In [8]:
union, intersection_2021, intersection_2022 = split_to_intersection_and_union(df_2021 = df_2021_5, df_2022=df_2022_5)

## handling contradictions

In [9]:
intersection_df = pd.merge(intersection_2021, intersection_2022, on=['id', 'redcap_event_name'], how='outer', suffixes=('_2021', '_2022'))

In [13]:
result2 = intersection_df.copy()

for key, value in merging_functions_by_columns_set.items():
    print(key, value)
    columns = columns_dict[key]
    for col in columns:
        result2[col] = result2.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
        result2 = result2.drop([f'{col}_2021', f'{col}_2022'], axis=1)

## iterate over the other columns
        
# validate
# i = columns.iloc[0]
# result2[(~result2[f'{i}_2022'].isna()) & (~result2[f'{i}_2021'].isna())][[i, f'{i}_2021', f'{i}_2022']]

diagnosis_binary <built-in function max>
complaint_binary <built-in function max>
timestamp <built-in function min>
date <built-in function min>
groups <built-in function max>
age_child_pre <built-in function max>
record_id take 2021
chameleon_binary_0_1 <built-in function max>
chameleon_binary_1_2 <built-in function min>
opening_psychothe_binary_0_1 <built-in function max>
opening_psychothe_binary_1_2 <built-in function min>
opening_medicine_binary_0_1 <built-in function max>
opening_medicine_binary_1_2 <built-in function min>
with_who_m <built-in function max>
with_who_f <built-in function max>


In [11]:
result2 = fix_wrong_groups(result2)

        id redcap_event_name
231  D2016      intake_arm_1


ValueError: 